# Part 1 : Defining the Recurrence Relation and Benchmarking the Runtime for growing Values of N  

In [1]:
def F(n):
    counter = 0
    firstnum = 0
    secondnum = 1
    newnum = 0
    
    while counter < n-1:
        newnum = firstnum +secondnum
        firstnum = secondnum
        secondnum = newnum
        counter = counter + 1
        
    return newnum

In [2]:
F(12)

144

In [1]:
print(int(9))

9


In [3]:
# Imports all the needed built in libraries
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

KeyboardInterrupt: 

In [ ]:
# Define a timer for python
import time 

class Timer:    
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.interval = self.end - self.start

In [ ]:
N = 100
# Makes a random square matrix with dimensions N+2 of type float64
utest = np.random.randint(10, size=(N+2,N+2)).astype(float)

In [ ]:
def diffusion_iteration(un):
    
    '''Definition of the recurrence relation and its boundary conditions
    Inputs:
    un: The original array that we need to apply the iteration to
    Outputs:
    u1: New array that the recurrance relation has been applied to'''
    
    N = len(un)
    
    u1 = un.copy()
    
    for i in range(1,N-1):
        for j in range(1,N-1):
            u1[i][j]=0.25*(un[i+1][j]+un[i-1][j]+un[i][j+1]+un[i][j-1])
            
                    
    return u1

In [ ]:
# Starts timer
with Timer() as t: 
# Prints the first interation of the recurrance relation
    print (diffusion_iteration(utest))
print("Time taken to apply diffusion iteration: {0}".format(t.interval))

## Code for Plot of N against Time 

In [ ]:
# Start timer for cell
starttotal = time.time()
#Empty times array
times = []
Nrange = 400
for x in range(Nrange):
    N=x
    # New array for every N value from 0 to Nrange
    u0 = np.random.randint(10, size=(N+2,N+2)).astype(float)
    with Timer() as t: 
        diffusion_iteration(u0)
    times.append(t.interval)
print("Total Time for Cell: ",time.time()-starttotal) 

In [ ]:
# x coordinates from 0 to Nrange
x = np.arange(Nrange)
plt.figure()
plt.plot(x,times,".")
plt.title('Runtime against N')
plt.xlabel('N')
plt.ylabel('Time/s')

## Complexity of runtime as N gets bigger

As the value of N gets larger the graph shows that the runtime of each iteration is also getting larger at a quadratic rate, which is expected, as the for loop is exectuting the function for $(N+2)^2$ values which is increasing with $N$. 


The main observation I made from the graph was when a close up of any points between 0 and 200 were taken; many points in this range had the same time. From my understanding this must be due to the insignificant difference in the times, and python is unable to differentiate them.

# Part 2: Implementing Numba

In [ ]:
from numba import njit, jit
N=100

In [ ]:
@njit
def diffusion_iteration_v2(un_v2):
    
    '''Definition of the recurrance relation and its boundary conditions
    Inputs:
    un_v2: The original array that we need to apply the iteration to
    Outputs:
    u1_v2: New array that the recurrance relation has been applied to'''
    
    N = len(un_v2)
    
    u1_v2 = un_v2.copy()
    
    for i in range(1,N-1):
        for j in range(1,N-1):
            u1_v2[i][j]=0.25*(un_v2[i+1][j]+un_v2[i-1][j]+un_v2[i][j+1]+un_v2[i][j-1])
            
                    
    return u1_v2

In [ ]:
# Starts timer
with Timer() as t: 
# Prints the first first interation of the recurrance relation
    print (diffusion_iteration_v2(utest))
print("Time taken to apply diffusion iteration: {0}".format(t.interval))

In [ ]:
# Start timer for cell
starttotal = time.time()
#Empty times array
times1 = []
for x in range(Nrange):
    N=x
    # New array for every N value from 0 to Nrange
    u0 = np.random.randint(10, size=(N+2,N+2)).astype(float)
    with Timer() as t: 
        diffusion_iteration_v2(u0)
    times1.append(t.interval)
print("Total time for cell: ",time.time()-starttotal)

In [ ]:
# x coordinates from 0 to Nrange
x = np.arange(Nrange)
plt.figure()
plt.plot(x,times1,".")
plt.title('Runtime against N')
plt.xlabel('N')
plt.ylabel('Time/s')

## Speed-up with Numba

As can be seen above implementing njit will take longer than pure python for 1 iteration, but, is significantly faster for 400 different iterations. njit, just-in time compiles Python code into fast machine code, this is useful for many for loops but will take longer for a single operation as the conversion is longer. This is further proven by the graph as most points take 0 seconds; the ones that take longer must be because numba is recompiling.

## Implementing Parallelisation

In [ ]:
from numba import njit, jit, prange
N=100

In [ ]:
@njit(parallel=True)
def diffusion_iteration_v3(un_v3):
    
    '''Definition of the recurrance relation and its boundary conditions
    Inputs:
    un_v3: The original array that we need to apply the iteration to
    Outputs:
    u1_v3: New array that the recurrance relation has been applied to'''
    
    N = len(un_v3)
    
    u1_v3 = un_v3.copy()
    
    for i in prange(1,N-1):
        for j in prange(1,N-1):
            u1_v3[i][j]=0.25*(un_v3[i+1][j]+un_v3[i-1][j]+un_v3[i][j+1]+un_v3[i][j-1])
            
                    
    return u1_v3

In [ ]:
# Starts timer
with Timer() as t: 
# Prints the first interation of the recurrance relation
    print (diffusion_iteration_v3(utest))
print("Time taken to apply diffusion iteration: {0}".format(t.interval))

In [ ]:
# Start timer for cell
starttotal = time.time()
#Empty times array
times2 = []
for x in range(Nrange):
    N=x
    # New array for every N value from 0 to Nrange
    u0 = np.random.randint(10, size=(N+2,N+2)).astype(float)
    with Timer() as t: 
        diffusion_iteration_v3(u0)
    times2.append(t.interval)
print("Total time for cell: ",time.time()-starttotal)

In my cpu (i5-8600k) I have 6 cores so my theoretical time should be 0.05 seconds for an Nrange of 400, but, it gives a very similar time to when parallelisation is false. I think this is due to the unnecessary use of extra cores as one core is as fast as the process can be done.

# Part 3: Visualisation

In [ ]:
N = 12
# Makes a random square matrix with dimensions N+2 of type float64
utest = np.random.randint(10, size=(N+2,N+2)).astype(float)

In [ ]:
@njit
def diffusion_iteration_v4(un_v4):
    
    '''Definition of the recurrance relation and its boundary conditions
    Inputs:
    un_v4: The original array that we need to apply the iteration to
    Outputs:
    u1_v4: New array that the recurrance relation has been applied to with new boundary conditions'''
    
    N = len(un_v4)
    
    u1_v4 = un_v4.copy()
    
    for i in range(1,N-1):
        for j in range(1,N-1):
                u1_v4[i][j]=0.25*(un_v4[i+1][j]+un_v4[i-1][j]+un_v4[i][j+1]+un_v4[i][j-1])
    
    #New boundary conditions
    u1_v4[0]= 0 * np.ones(N)
    u1_v4[N-1]= 0 * np.ones(N)
    
    for i in range(N):
        u1_v4[i][0] = 0
    for i in range (N):
        u1_v4[i][N-1] = 0
                    
    return u1_v4

In [ ]:
#Plots a static representation of u1 for utest 
plt.figure()
plt.imshow(diffusion_iteration_v4(utest))

In [ ]:
def diffusion_iteration_number(un,n):

    '''Definition of the recurrance relation and its boundary conditions
    Inputs:
    un: The original array that we need to apply the iteration to
    n: the number of iterations
    Outputs:
    num_iterations: New array that the recurrance relation has been applied to n times'''
    
    num_iterations = []
    for x in range(n):
        un = diffusion_iteration_v4(un)
        num_iterations.append(un)
    return num_iterations

In [ ]:
arr_iterations = diffusion_iteration_number(utest,50)

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
from matplotlib import animation
fig = plt.figure()
ax = plt.axes(xlim=(0, N+1), ylim=(0, N+1))

#Set a to first array in list 
a = arr_iterations[0]
im = plt.imshow(a,interpolation='none')

# initialization function: plot the background of each frame    
def init():
    for i in range(len(arr_iterations)):
        im.set_data(arr_iterations[i])
    return [im]


# animation function.  This is called sequentially
def animate(i):
    #Gets current array from im and sets next array to a 
    a = im.get_array()
    a = arr_iterations[i]   
    im.set_array(a)
    return [im]

#Generates the animation and sets each cycle to 8 seconds
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=52, interval=150, blit=True)

The animated plot shows how the outside zeros converge inwards as more iterations are applied to the same matrix. The corners are the first values to drop to 0 as the average is calculated by dividing 2 numbers by 4.

# Part 4: Advanced Problem

In [ ]:
N = 12
# Makes a random square matrix with dimensions N+2 of type float64
utest = np.random.randint(10, size=(N+2,N+2)).astype(float)

In [ ]:
@njit(parallel=True)
def diffusion_iteration_v5(un_v5,c_i):
    
    '''Definition of the recurrance relation and its boundary conditions
    Inputs:
    un_v5: The original array that we need to apply the iteration to
    c_i: A list of all the diagonal points that will be kept the same
    Outputs:
    u1_v5: New array that the recurrance relation has been applied to with the additional boundary condition'''
    
    N = len(un_v5)
    u1_v5 = un_v5.copy()
    
    for i in prange(1,N-1):
        for j in range(1,N-1):
            
            for x in range(len(c_i)):
                u1_v5[c_i[x]][c_i[x]] = un_v5[c_i[x]][c_i[x]]
                
            u1_v5[i][j]=0.25*(un_v5[i+1][j]+un_v5[i-1][j]+un_v5[i][j+1]+un_v5[i][j-1])
     
    
    #New boundary conditions
    u1_v5[0]= 0 * np.ones(N)
    u1_v5[N-1]= 0 * np.ones(N)
    
    for i in range(N):
        u1_v5[i][0]= 0
    for i in range (N):
        u1_v5[i][N-1]= 0
                
                 
    return u1_v5

In [ ]:
def diffusion_iteration_number1(un,c_i,n):

    '''Definition of the recurrance relation and its boundary conditions
    Inputs:
    un: The original array that we need to apply the iteration to
    n: The number of iterations
    c_i: A list of all the diagonal points that will be kept the same
    Outputs:
    num_iterations: New array that the recurrance relation has been applied to n times'''
    
    num_iterations = []
    for x in range(n):
        un = diffusion_iteration_v5(un,c_i)
        num_iterations.append(un)
    return num_iterations

In [ ]:
import random
# Generates an array with 5 random diagonals that stay constant
con_diag = diffusion_iteration_number1(utest,random.sample(range(1, N), 5),500)

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
from matplotlib import animation
fig1 = plt.figure()
ax1 = plt.axes(xlim=(0, N+1), ylim=(0, N+1))

#Set a to first array in list 
a1 = con_diag[0]
im1 = plt.imshow(a,interpolation='none')

# initialization function: plot the background of each frame    
def init1():
    for i in range(len(con_diag)):
        im1.set_data(con_diag[i])
    return [im1]


# animation function.  This is called sequentially
def animate1(i):
    #Gets current array from im and sets next array to a 
    a1 = im1.get_array()
    a1 = con_diag[i]   
    im1.set_array(a1)
    return [im1]

#Generates the animation and sets each cycle to 8 seconds
anim1 = animation.FuncAnimation(fig1, animate1, init_func=init1,
                               frames=52, interval=150, blit=True)

This animated plot shows how zeros converge inwards when 5 random numbers in the diagonal are fixed. It shows that high fixed points will increase the average of surrounding points so the zeros will not converge to them.